![](https://images.unsplash.com/photo-1544954412-78da2cfa1a0c?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2071&q=80)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy()

In [ ]:
dandelion = "../input/dandelionimages/Images/dandelion/"
other = "../input/dandelionimages/Images/other/"

In [ ]:
dataset_length = min(len(os.listdir(dandelion)),len(os.listdir(other)))
dataset_length

In [ ]:
p = [{"name":dandelion + os.listdir(dandelion)[i],"label":1} for i in range(dataset_length)]
q = [{"name":other + os.listdir(other)[i],"label":0} for i in range(dataset_length)]

In [ ]:
data_1 = pd.DataFrame(p)
data_2 = pd.DataFrame(q)

In [ ]:
frames = [data_1, data_2]
df = pd.concat(frames)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

In [ ]:
_, axs = plt.subplots(5, 5, figsize=(12, 12))
axs = axs.flatten()
img_ = [df.name.values[np.random.randint(len(df.name.values))] for i in range(25)]
for img, ax in zip(img_, axs):
    ax.imshow(Image.open(img))
plt.show()

In [ ]:
#hyperparamets
batch_size = 8
dropout = 0.2
lr = 1e-3
decay_steps = 100
decay_rate = 0.96
epochs = 20

In [ ]:
def get_image(path):
    image = tf.image.decode_jpeg(tf.io.read_file(path),channels=3)
    image = tf.cast(tf.image.resize_with_pad(image,299,299),dtype=tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

In [ ]:
def process_dataset(path, label):
    return get_image(path), label

In [ ]:
def get_dataset(x, y=None):
    if y is not None:
        ds = tf.data.Dataset.from_tensor_slices((x, y))
        return ds.map(process_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    else:
        ds = tf.data.Dataset.from_tensor_slices(x)
        return ds.map(get_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(df.name.values,df.label.values,test_size=0.02,random_state=5)

In [ ]:
train_ = get_dataset(x_train,y_train) 
valid_ = get_dataset(x_valid,y_valid) 

In [ ]:
# with strategy.scope():
model_in = tf.keras.applications.InceptionV3(weights='imagenet')
model_in.trainable = False
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(229, 229, 3)),
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
    model_in,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(dropout, name='top_dropout'),
    tf.keras.layers.Dense(32, activation='elu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)
lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [ ]:
history=model.fit(train_,validation_data=valid_,
                  epochs=epochs,verbose=1,callbacks=[early_stop,lr],use_multiprocessing=True, workers=-1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

Thanks For Watching notebook , Enjoy Machine learning 🤗